In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Data/consolidatedDataForPackage.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])
df = data[data['subjectId']=='1636-69-032']
df = df.reset_index(drop=True)

In [3]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [6]:
def congaN(df, n):
    if (not(n==1 or n==2 or n==4)):
        print("WARNING: Standard range for CONGA n values are 1,2 or 4 hours. CONGA measure might be  unreliable if observations are more than 4h apart.")
    
    day = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    day = day.round("d")
    day = day.days

    df = df.set_index(['Display Time'])
    t = str(n*3600)+'s'
    gv = df['GlucoseValue'].resample(t).first()

    k = len(gv)

    frame = pd.DataFrame()
    frame['GV'] = gv
    frame['Dt'] = frame['GV'] - frame['GV'].shift(+1)
    frame = frame.fillna(0)

    dBar = sum(frame['Dt']) / k

    s = 0
    for i in frame['Dt']:
        s += (i-dBar)**2

    conga = math.sqrt(s/(k-1))

    return round(conga/day, 2)

In [7]:
conga_1 = congaN(df, 1)
conga_2 = congaN(df, 2)
conga_4 = congaN(df, 4)

In [8]:
conga_1,conga_2,conga_4

(2.07, 2.67, 3.05)